# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
import re
import select
import sys
import json
import time
from itertools import islice
from concurrent.futures import ThreadPoolExecutor
from typing import Optional, Generator, Callable, Dict, Any
import requests
import schedule
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
def get_soup(session: requests.Session,
             base_url: str) -> Optional[BeautifulSoup]:
    """
    Fetch HTML content from a URL and parse it into a BeautifulSoup
    object.

    Args:
        session (requests.Session): The requests session to use for the
                                    HTTP request.
        base_url (str): The URL to fetch HTML content from.

    Returns:
        Optional[BeautifulSoup]:
            - BeautifulSoup object if request is successful
            - None if an error occurs during HTML parsing

    Raises:
        requests.RequestException: If there's an issue with the HTTP
                                   request (connection error, timeout,
                                   HTTP error, etc.)

    Note:
        HTTP-related exceptions are propagated to the caller for
        handling.
        Returns None only for HTML parsing issues, not for HTTP errors.

    Example:
        >>> import requests
        >>> session = requests.Session()
        >>> try:
        ...     soup = get_soup(session, "https://example.com")
        ...     if soup:
        ...         title = soup.find('title')
        ...         print(title.text)
        ... except requests.RequestException as e:
        ...     print(f"Request failed: {e}")
        >>> session.close()
    """
    response = session.get(base_url, timeout=20)
    response.raise_for_status()
    return BeautifulSoup(response.content, 'lxml')


def get_books_links(
        session: requests.Session,
        base_url: str,
        _raw_url: Optional[str] = None) -> Generator[str, None, None]:
    """
    Generates book links from an online catalog.

    The function iterates through catalog pages, extracts book links
    and handles pagination. Automatically adjusts paths by adding
    'catalogue/' to relative URLs when necessary.

    Args:
        session (requests.Session): The requests session to use for HTTP
                                    requests
        base_url (str): Base catalog URL to start parsing from
        _raw_url (Optional[str]): Raw URL for edge case testing.
                                  If not provided, uses base_url + '/'

    Yields:
        str: Full URLs of book links

    Note:
        _raw_url is needed for edge case testing when starting from
        specific pages. See usage example.

    Examples:
        Basic usage:
            >>> with requests.Session() as session:
            ...     for book_link in get_books_links(
                        session, "http://books.toscrape.com"
                    ):
            ...         print(book_link)

        Edge case - starting from specific page:
            >>> b_url = ('https://books.toscrape.com/'
                         +'catalogue/page-31.html')
            >>> r_url = 'https://books.toscrape.com/'
            >>> with requests.Session() as session:
            ...     res = list(get_books_links(session,
                                               base_url=b_url,
                                               _raw_url=r_url))
            ...     print(f"Found {len(res)} books start with page 31")
    """
    in_catalogue: Callable[[str], str] = lambda link: (
        '' if 'catalogue' in link else 'catalogue/'
    )

    if not _raw_url:
        _raw_url = base_url.rstrip('/') + '/'

    while base_url:
        soup = get_soup(session, base_url)
        if not soup:
            break

        for link in soup.find_all('a', title=True):
            href = link.get('href', '')
            if href:
                yield _raw_url + in_catalogue(href) + href

        next_page = soup.find('li', class_='next')
        if next_page and next_page.a:
            next_page = next_page.a['href']
            base_url = _raw_url + in_catalogue(next_page) + next_page
        else:
            base_url = None

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
def get_book_data(session: requests.Session, book_url: str) -> Dict[str, Any]:
    """
    Extracts detailed information about a book from its product page.

    This function parses the HTML content of a book page and extracts
    various details including title, price, availability, rating,
    description, and product information such as UPC, product type,
    tax details, etc.

    Args:
        session (requests.Session): The requests session to use for HTTP
                                    requests
        book_url (str): The URL of the book's product page to scrape.

    Returns:
        Dict[str, Any]: A dictionary containing book data with the
        following structure:
            - 'title' (str): Book title
            - 'price' (str): Book price in format '£X.XX'
            - 'in stock' (str): Availability information
            - 'rating' (int): Numeric rating from 1 to 5
            - 'product description' (str): Book description text
            - 'product information' (Dict[str, str]): Detailed product
              info including:
                    'UPC'; 'product Type'; 'price (excl. tax)';
                    'price (incl. tax)'; 'tax'; 'availability';
                    'number of reviews'

    Note:
        Returns an empty dictionary if the page cannot be loaded or
        parsed.
        Converts star rating classes ('One', 'Two', etc.) to numeric
        values.
        Some fields may be missing or empty if the corresponding HTML
        elements are not found.
        The 'product description' field may be an empty string.

    Raises:
        No exceptions are raised externally; all errors are handled
        internally by returning an empty dictionary. Internal parsing
        errors may occur if the HTML structure differs from expected.

    Example:
        >>> session = requests.Session()
        >>> book_data = get_book_data(
        ...     session,
        ...     "http://books.toscrape.com/catalogue/"
        ...     + "a-light-in-the-attic_1000/index.html"
        ... )
        >>> session.close()
        >>> print(book_data['title'])
        'A Light in the Attic'
        >>> print(book_data['price'])
        '£51.77'
        >>> print(book_data['rating'])
        3
    """
    book_data: Dict[str, Any] = {}
    ratings: Dict[str, int] = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4,
                               'Five': 5}
    re_price: re.Pattern = re.compile(r'£\d+\.\d{2}')
    re_availability: re.Pattern = re.compile(r'\((.*?)\)')
    soup: Optional[BeautifulSoup] = get_soup(session, book_url)

    p_main: BeautifulSoup
    info_table: BeautifulSoup
    desc: BeautifulSoup
    if soup:
        p_main = soup.find('div', class_='col-sm-6 product_main')
        info_table = soup.find('table')
        desc = soup.find('div', id='product_description')

    if soup and p_main and info_table:
        book_data = {
            'title': p_main.find('h1').text.strip(),
            'price': (re_price
                      .search(p_main
                              .find('p', class_='price_color')
                              .text)
                      .group()),
            'in stock': re_availability.search(p_main.text).group(1),
            'rating': (
                ratings.get(
                    p_main.find('p', class_='star-rating')['class'][-1],
                    None)
            ),
            'product description': (desc
                                    .find_next_sibling('p')
                                    .text
                                    .strip()) if desc else '',
            'product information': {
                'UPC': info_table.find_all('tr')[0].td.text,
                'product Type': info_table.find_all('tr')[1].td.text,
                'price (excl. tax)': (
                    re_price.search(info_table.find_all('tr')[2].td.text)
                    .group()),
                'price (incl. tax)': (
                    re_price.search(info_table.find_all('tr')[3].td.text)
                    .group()),
                'tax': (re_price
                        .search(info_table.find_all('tr')[4].td.text)
                        .group()),
                'availability': (re_availability
                                 .search(info_table.find_all('tr')[5].td.text)
                                 .group(1)),
                'number of reviews': info_table.find_all('tr')[6].td.text
            }
        }

    return book_data

In [4]:
s = requests.Session()
book = get_book_data(s, 'https://books.toscrape.com/catalogue/'
              + 'a-light-in-the-attic_1000/index.html')
s.close()
print(
    'Product: ' + book['product information']['product Type'],
    'Title: ' + book['title'],
    'Price: ' + book['price'],
    'Rating: ' + str(book['rating']),
    'Available: ' + book['in stock'],
    sep='\n',
)

Product: Books
Title: A Light in the Attic
Price: £51.77
Rating: 3
Available: 22 available


## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(
        base_url: str = '',
        _raw_url: Optional[str] = None,
        batch_size: int = 200,
        is_save: bool = False,
        file_name: str = './books_data.txt') -> Dict[str, Any]:
    """
    Scrapes book data from an online catalog using parallel processing.

    This function iterates through all pages of a book catalog, extracts
    book URLs, and then concurrently scrapes detailed information for
    each book. It features progress tracking, batch processing,
    and optional data persistence.

    Args:
        base_url (str): The starting URL of the book catalog to scrape.
        _raw_url (Optional[str]): Internal parameter for edge case
                                  handling of URL formatting.
        batch_size (int): Number of books to process in each batch
                          (default: 200).
        is_save (bool): If True, saves the scraped data to a file
                        (default: False).
        file_name (str): Path where to save the resulting file when
                         is_save is True
                         (default: './artifacts/books_data.txt').

    Returns:
        Dict[str, Any]: A dictionary where keys are book URLs
                        and values are dictionaries containing
                        detailed book information.
                        Returns empty dict if no books found.

    Note:
        Uses ThreadPoolExecutor for concurrent scraping.
        Includes progress visualization with tqdm.
        Saves data only if is_save=True and books data is available.

    Example:
        >>> # Basic usage
        >>> books = scrape_books("http://books.toscrape.com")
        >>> print(f"Scraped {len(books)} books")

        >>> # With saving to file
        >>> books = scrape_books(
        ...     base_url="http://books.toscrape.com",
        ...     is_save=True
        ... )
        >>> # Data will be saved to 'books_data.txt'

        >>> # Edge case - starting from specific page:
        >>> books = scrape_books(
        ...     base_url=('https://books.toscrape.com'
        ...              + '/catalogue/page-31.html'),
        ...     _raw_url='https://books.toscrape.com/',
        ...     is_save=True
        ... )
        >>> # Data will be saved to 'books_data.txt'
    """
    books: Dict[str, Any] = {}

    try:
        with requests.Session() as session:
            links = get_books_links(session, base_url, _raw_url)
            soup = get_soup(session, base_url)

            if soup:
                strong_tags = soup.find_all('strong')
                total = int(strong_tags[0].text) - int(strong_tags[1].text) + 1

                with tqdm(total=total, desc='Scrape books', ncols=100) as pbar:
                    with ThreadPoolExecutor(max_workers=70) as executor:
                        while True:
                            batch = list(islice(links, batch_size))
                            if not batch:
                                break

                            books.update(
                                zip(batch, executor.map(
                                    lambda url: get_book_data(session, url),
                                    batch))
                            )
                            pbar.update(len(batch))

    except requests.RequestException as e:
        print(f"Error during scraping {base_url}: {e}")

    finally:
        if is_save:
            if books:
                with open(file_name, 'w', encoding='utf-8') as f:
                    json.dump(books, f, ensure_ascii=False, indent=4)
                print(f"The data has been saved to file '{file_name}'!")
            else:
                print("No data to save - books dictionary is empty")

    return books

In [6]:
# Проверка работоспособности функции
_raw_url = 'https://books.toscrape.com/'
res = scrape_books(_raw_url, is_save=True) # Допишите ваши аргументы
print('Is dict:', type(res) == dict)
print('1000 elements:', len(res) == 1000)

res.get('https://books.toscrape.com/'
        +'catalogue/tipping-the-velvet_999/index.html')

Scrape books: 100%|█████████████████████████████████████████████| 1000/1000 [01:06<00:00, 14.99it/s]

The data has been saved to file './books_data.txt'!
Is dict: True
1000 elements: True


{'title': 'Tipping the Velvet',
 'price': '£53.74',
 'in stock': '20 available',
 'rating': 1,
 'product description': '"Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s "Erotic and absorbing...Written with starling power."--"The New York Times Book Review " Nan King, an oyster girl, is captivated by the music hall phenomenon Kitty Butler, a male impersonator extraordinaire treading the boards in Canterbury. Through a friend at the box office, Nan manages to visit all her shows and finally meet her heroine. Soon after, she becomes Kitty\'s dresser and the two head for the bright lights of Leicester Square where they begin a glittering career as music-hall stars in a

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
schedule.every().day.at("19:00:00", 'Europe/Moscow').do(
    scrape_books, base_url='https://books.toscrape.com/', is_save=True)

while True:
    print('Task Scheduler has started.')
    schedule.run_pending()
    time.sleep(1)

Task Scheduler has started.


Scrape books: 100%|█████████████████████████████████████████████| 1000/1000 [01:06<00:00, 15.06it/s]

The data has been saved to file './books_data.txt'!


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
# ! pytest tests/test_scraper.py  # очень лениво =( поэтому не написал

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```